In [1]:
import igraph as ig
import numpy as np
import louvain
import time

/var/folders/b7/w5j5st2x5d9f77725jtxfcqh0000gn/T/ipykernel_38045/3988221483.py:3: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


In [2]:
print("Construct the graph from the txt file (edgelist)")

input = np.loadtxt("../data/PowerUS/ElistPowerUS.txt", dtype='i', delimiter=' ');
dim = input.shape;
N = input.max();
g = ig.Graph();
g.add_vertices(N);
for i in range(0, dim[0]):
    g.add_edges([(input[i,0]-1,input[i,1]-1)]);
ig.summary(g)

Construct the graph from the txt file (edgelist)
IGRAPH U--- 4941 6594 -- 


In [3]:
print("Largest connected component")
largest = g.clusters().giant();
ig.summary(largest)

Largest connected component
IGRAPH U--- 4941 6594 -- 


/var/folders/b7/w5j5st2x5d9f77725jtxfcqh0000gn/T/ipykernel_38045/2916332679.py:2: DeprecationWarning: Graph.clusters() is deprecated; use Graph.connected_components() instead
  largest = g.clusters().giant();


In [4]:
print("Running Louvain 1 times in C++")
start_time = time.time()

part = louvain.find_partition(largest, louvain.ModularityVertexPartition);

        
print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part.modularity)
print("Sizes of the groups:", part.sizes())

Running Louvain 1 times in C++
--- 0.14903879165649414 seconds ---
Modularity : 0.9368100811125161
Sizes of the groups: [211, 207, 203, 198, 195, 177, 172, 168, 167, 164, 164, 162, 156, 153, 148, 148, 143, 136, 131, 131, 130, 115, 111, 105, 102, 101, 101, 99, 95, 74, 72, 71, 71, 69, 66, 61, 61, 47, 31, 25]


In [5]:
print("Running Louvain 1000 times in C++")
start_time = time.time()

m_best =0;
part_best = louvain.find_partition(largest, louvain.ModularityVertexPartition);


for n in range(1000):
    part = louvain.find_partition(largest, louvain.ModularityVertexPartition);
    m = part.modularity;
    if m_best<m:
        m_best = m;
        part_best = part;
        

print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part_best.modularity)
print("Sizes of the groups:", part_best.sizes())

Running Louvain 1000 times in C++


KeyboardInterrupt: 

In [ ]:
len(part_best)

40

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

membership =part_best.membership;
H = lil_matrix((part_best.n,len(part_best.sizes())));
i = 0;
for n in membership:
    H[i,n] =1.;
    i = i+1;
    
e_list = largest.get_edgelist();
e_list = np.array(e_list);

A = lil_matrix((part_best.n,part_best.n), dtype=np.int8);

for n in range(len(e_list)):
    A[e_list[n,0],e_list[n,1]] = 1;

A = A+lil_matrix.transpose(A);
deg = A.sum(0)


L_n = lil_matrix((part_best.n,part_best.n));

for n in range(len(e_list)):
    L_n[e_list[n,0],e_list[n,1]] = -1./np.sqrt(deg[0,e_list[n,0]]*deg[0,e_list[n,1]]);
    
L_n = L_n+lil_matrix.transpose(L_n);


for a in range(part_best.n):
    L_n[a,a] = 1.;
            

T =(lil_matrix.transpose(H)*L_n*H);
d = np.array(T.diagonal());
n_cut = np.sum(d);
print('Normalized Cut')
print(n_cut)

/home/fanuel/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Normalized Cut
579.307921805
